In [15]:
#Import packages here
import pandas as pd
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import re
import concurrent.futures

In [10]:
df = pd.read_csv('microbigge_ecoli.csv')

In [11]:
df.shape

(23295, 18)

In [12]:
df['Type'].value_counts(normalize=True)

Type
AMR          0.504958
VIRULENCE    0.426830
STRESS       0.068212
Name: proportion, dtype: float64

In [16]:
# Define a function to fetch the DNA sequence from the NCBI website, include messages for status updates and if an error arises
def fetch_dna_sequence(contig, start, stop, retries=3, timeout=10):
    url = f"https://www.ncbi.nlm.nih.gov/nuccore/{contig}?from={start}&to={stop}"
    for _ in range(retries):
        try:
            response = requests.get(url, timeout=timeout)
            response.raise_for_status()
            return response.text
        except requests.RequestException as e:
            print(f"Error fetching {url}: {e}")
    return None

In [6]:
# Define a function to extract the nucleotide sequence from the ORIGIN section
def extract_origin_sequence(data):
    if data:
        lines = data.splitlines()
        origin_started = False
        sequence = []
        for line in lines:
            if line.startswith("ORIGIN"):
                origin_started = True
            elif origin_started:
                if line.startswith("//"):
                    break
                sequence.append(''.join(filter(str.isalpha, line)))
        return ''.join(sequence)
    return ""

In [7]:
# Apply the functions to each row in the DataFrame and add a new column for the DNA sequence
def get_dna_sequence(row):
    data = fetch_dna_sequence(row['Contig'], row['Start'], row['Stop'])
    sequence = extract_origin_sequence(data)
    return sequence

In [14]:
def process_row(row):
    return get_dna_sequence(row)

In [17]:
# Apply the function with parallel processing to expedite it
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(tqdm(executor.map(process_row, df.to_dict('records')), total=len(df)))

#Add results of concurrent.futures work to the DataFrame
df['full_sequence'] = results

# Save the result as a new CSV file
df.to_csv('fullseq_microbigge_ecoli.csv', index=False)



  8%|▊         | 1926/23295 [05:40<48:56,  7.28it/s]  

Error fetching https://www.ncbi.nlm.nih.gov/nuccore/AAVVQG010000001.1?from=202121&to=203542: HTTPSConnectionPool(host='www.ncbi.nlm.nih.gov', port=443): Read timed out. (read timeout=10)


  8%|▊         | 1926/23295 [09:04<48:56,  7.28it/s]

Error fetching https://www.ncbi.nlm.nih.gov/nuccore/AAVVQJ010000003.1?from=177596&to=180742: HTTPSConnectionPool(host='www.ncbi.nlm.nih.gov', port=443): Read timed out. (read timeout=10)
Error fetching https://www.ncbi.nlm.nih.gov/nuccore/AAVVQC010000001.1?from=105933&to=107354: HTTPSConnectionPool(host='www.ncbi.nlm.nih.gov', port=443): Read timed out. (read timeout=10)
Error fetching https://www.ncbi.nlm.nih.gov/nuccore/AAVVQF010000002.1?from=245139&to=246158: HTTPSConnectionPool(host='www.ncbi.nlm.nih.gov', port=443): Read timed out. (read timeout=10)
Error fetching https://www.ncbi.nlm.nih.gov/nuccore/AAVVQE010000001.1?from=144181&to=145602: HTTPSConnectionPool(host='www.ncbi.nlm.nih.gov', port=443): Read timed out. (read timeout=10)
Error fetching https://www.ncbi.nlm.nih.gov/nuccore/AAVVPZ010000001.1?from=46840&to=47373: HTTPSConnectionPool(host='www.ncbi.nlm.nih.gov', port=443): Read timed out. (read timeout=10)
Error fetching https://www.ncbi.nlm.nih.gov/nuccore/AAVVQA010000001

 13%|█▎        | 3033/23295 [12:32<46:45,  7.22it/s]    

...

 84%|████████▍ | 19677/23295 [1:39:40<6:04:37,  6.05s/it]

Error fetching https://www.ncbi.nlm.nih.gov/nuccore/AASNKK010000001.1?from=268757&to=270115: HTTPSConnectionPool(host='www.ncbi.nlm.nih.gov', port=443): Read timed out. (read timeout=10)


 84%|████████▍ | 19678/23295 [1:39:42<5:08:21,  5.12s/it]

Error fetching https://www.ncbi.nlm.nih.gov/nuccore/AASNKD010000002.1?from=90981&to=92402: HTTPSConnectionPool(host='www.ncbi.nlm.nih.gov', port=443): Read timed out. (read timeout=10)


100%|██████████| 23295/23295 [1:50:20<00:00,  3.52it/s]  
